In [ ]:
!pip install mysql-connector 

In [ ]:
import mysql.connector
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

with open ('config.json') as f:
    configData = json.load(f)

engine = create_engine(configData['engine_url'])

mydb = mysql.connector.connect(
        host = configData['host'],
        user = configData['user'],
        password = configData['password'],
        database = configData['database']
    )

df = pd.read_excel('data.xlsx')

# Filter dataframe

df = df[df['STATUS'] == 'REDEEMED']
df = df[df['STATE'] == 'New Jersey']

print (len(df))

6950


In [ ]:
column = {
    "MANAGING_COMPANY": "FUND",
    "CERTIFICATE": "CERTIFICATE",
    "INVESTMENT_DATE": "INVESTMENT DATE",
    "ORIGINAL_LIEN_AMOUNT": "BEGINNING BALANCE",
    "ORIGINAL_LIEN_INTEREST": "BEGINNING BALANCE DEDUCTED INTEREST",
    "ORIGINAL_LIEN_EFFECTIVE_DATE": "BEGINNING BALANCE EFFECTIVE DATE",
    "ORIGINAL_LIEN_INTERVAL": "FEES: INTERVAL", 
    "PREMIUM_AMOUNT": "PREMIUMS",
    "PREMIUM_INTEREST": "FEES: INTEREST", 
    "PREMIUM_EFFECTIVE_DATE": "FEES: EFFECTIVE DATE",
    "PREMIUM_INTERVAL": "FEES: INTERVAL",
    "TSRID": "TSRID",
    "STATE": "STATE",
    "COUNTY": "COUNTY",
    "MUNICIPALITY": "MUNICIPALITY",
    "PARCEL_ID": "PARCEL",
    "OWNER_NAME_CURRENT_OWNER": "PROPERTY OWNER",
    "OWNER_ADDRESS": "OWNER ADDRESS",
    "OWNER_CITY_STATE_ZIP": "OWNER CITY STATE ZIP",
    "HOMESTEAD_EXEMPTION": "HOMESTEAD EXEMPTION",
    "TOTAL_ASSESSED_VALUE": "ASSESSED VALUE",
    "TOTAL_MARKET_VALUE": "TOTAL MARKET VALUE",
    "APN": "APN",
    "LOCATION_FULL_STREET_ADDRESS": "OWNER ADDRESS",
    "LOCATION_CITY": "LOCATION CITY",
    "LOCATION_ZIP": "ZIP CODE",
    "LONGITUDE": "LONGITUDE",
    "LATITUDE": "LATITUDE",
    "OWNER_OCCUPIED": "OWNER OCCUPIED",
    "COUNTY_LAND_USE": "COUNTY LAND USE",
    "COUNTY_LAND_USE_DESC": "COUNTY LAND USE DESC",
    "STANDARDIZED_LAND_USE": "STANDARDIZED LAND USE",
    "LOT_SIZE": "LOT SIZE",
    "LOT_SIZE_UNIT": "LOT SIZE UNIT",
    "ZONING": "ZONING",
    "BUILDING_CLASS": "BUILDING CLASS",
    "YEAR_BUILT": "YEAR BUILT",
    "NO_OF_STORIES": "NO OF STORIES",
    "NO_OF_UNITS": "NO OF UNITS",
    "ASSESSMENT_YEAR": "ASSESSMENT YEAR",
    "LATEST_SALE_DATE": "LATEST SALE DATE",
    "LATEST_SALE_PRICE": "LATEST SALE PRICE",
    "LATEST_ARMS_LENGTH_SALE_DATE": "LATEST ARMS LENGTH SALE DATE",
    "LATEST_ARMS_LENGTH_SALE_PRICE": "LATEST ARMS LENGTH SALE PRICE",
    "PRIOR_ARMS_LENGTH_SALE_DATE": "PRIOR ARMS LENGTH SALE DATE",
    "PRIOR_ARMS_LENGTH_SALE_PRICE": "PRIOR ARMS LENGTH SALE PRICE",
    "LOAN1_AMOUNT": "LOAN1 AMOUNT",
    "LOAN1_DUE_DATE": "LOAN1 DUE DATE",
    "LOAN1_TYPE": "LOAN1 TYPE",
    "LOAN2_AMOUNT": "LOAN2 AMOUNT",
    "LEGAL_CITY": "LEGAL CITY",
    "LEGAL_BLOCK": "LEGAL BLOCK",
    "LEGAL_LOT_NUMBER": "LEGAL LOT NUMBER",
    "QUALIFIER": "QUALIFIER",
    "LEGAL_SECTION": "LEGAL SECTION",
    "LEGAL_UNIT": "LEGAL UNIT",
    "LEGAL_SUBDIVISION_NAME": "LEGAL SUBDIVISION NAME",
    "LEGAL_TRACT_NUMBER": "LEGAL TRACT NUMBER",
    "LEGAL_SECTION_TOWNSHIP_RANGE_MERIDIAN": "LEGAL SECTION TOWNSHIP RANGE MERIDIAN",
    "LEGAL_BRIEF_DESC": "LEGAL BRIEF DESC",
    "SUBJECT_FOUND_COUNT": "SUBJECT FOUND COUNT",
    "ENV_RISK": "ENV RISK",
    "TAG_KEEP_MAYBE_REMOVE_VIEWED": "TAG (KEEP, MAYBE, REMOVE, VIEWED)",
    "GRADE_A_B_C_D_F": "GRADE (A, B, C, D, F)",
    "GROUP_1_2_3_4_5": "GROUP (1, 2, 3, 4, 5)",
    "NOTES": "notes",
    "STATUS": "Status",
    "CREATE_DATE": "Created Date",
    "LAST_MODIFY_DATE": "Last Modified Date",
    "UNIQUE_ID": "REFERENCE ID",
}
column_dict = dict((v,k) for k,v in column.items())
req_cols = list(column_dict.values())

df1 = df.copy()
df1 = df1[df1['FEES: CATEGORY'] == 'Beginning Balance']

df1 = df1.rename(columns=column_dict)
df1 = df1[df1.columns[df1.columns.isin(req_cols)]]
df1['ORIGINAL_LIEN_INTERVAL'] = df1['PREMIUM_INTERVAL']
df1['LOAN1_DUE_DATE'] = df1['LOAN1_DUE_DATE'].fillna("1994-01-01", inplace = True)

In [ ]:
# df1 = df1[df1['PARCEL_ID'] == '10 00263-0000-00029']

date_cols = ['ORIGINAL_LIEN_EFFECTIVE_DATE', 'PREMIUM_EFFECTIVE_DATE',  'LATEST_SALE_DATE', 'LOAN1_DUE_DATE']
for i in date_cols:
  df1[i] = pd.to_datetime(df1[i]).dt.strftime("%Y-%m-%d").astype(str)
df1['STATUS'] = 10
df1 = df1[df1['TSRID'].isnull() == False]
# df1


df1.to_sql('PARCELS', engine, if_exists='append', index=False)

# df1.to_excel("test.xlsx")

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:1685: UserWarning: The provided table name 'PARCELS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [ ]:
# ADD SUBS
# Make sure to delete beg. balance and preimumn from the table

In [ ]:
column.update({
    "CATEGORY": "FEES: CATEGORY",
    "AMOUNT": "FEES: AMOUNT",	
    "DESCRIPTION": "FEES: DESCRIPTION",
    "EFFECTIVE_END_DATE": "FEES: END DATE"
})

column_dict = dict((v,k) for k,v in column.items())
req_cols = list(column_dict.values())

df2 = df.copy()
df2 = df2.rename(columns=column_dict)

# df2 = df2[df2['PARCEL_ID'] == '10 00263-0000-00029']

df2 = df2[['UNIQUE_ID', 'CATEGORY', 'AMOUNT', 'DESCRIPTION', 'PREMIUM_EFFECTIVE_DATE', 'PREMIUM_INTERVAL', 'PREMIUM_INTEREST', 'EFFECTIVE_END_DATE']]
df2 = df2[(df2['CATEGORY'] == 'Subsequent Tax') | (df2['CATEGORY'] == 'Beginning Balance') | (df2['CATEGORY'] == 'Premium')]

# Update category to correct number
category_dictionary ={'Subsequent Tax' : 3, 'Beginning Balance' : 1, 'Premium' : 2}
df2['CATEGORY'] = df2['CATEGORY'].map(category_dictionary)


df2['PREMIUM_EFFECTIVE_DATE'] = pd.to_datetime(df2['PREMIUM_EFFECTIVE_DATE']).dt.strftime("%Y-%m-%d").astype(str)
df2['EFFECTIVE_END_DATE'] = pd.to_datetime(df2['EFFECTIVE_END_DATE']).dt.strftime("%Y-%m-%d").astype(str)
df2['DESCRIPTION'] = df2['DESCRIPTION'].astype(str)
df2['DESCRIPTION'] = df2['DESCRIPTION'].replace('nan','')
df2.columns = ['UNIQUE_ID', 'CATEGORY', 'AMOUNT', 'DESCRIPTION', 'EFFECTIVE_DATE', 'INTEREST_ACC_INTERVAL', 'INTEREST', 'EFFECTIVE_END_DATE']

# Add to database
df2.to_sql('FEES', engine, if_exists='append', index=False)





/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:1685: UserWarning: The provided table name 'FEES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [ ]:
unique_parcel = df2['UNIQUE_ID'].unique()

# Get effective end date to update beg. balance and premium
# Run only to update beg. balance and premium, not usefull
# for i in unique_parcel:
#   test_df = df2[df2['UNIQUE_ID'] == i]
#   eff_end_date = test_df.iloc[0]['EFFECTIVE_END_DATE']
#   sql = "SET SQL_SAFE_UPDATES = 0; UPDATE FEES SET EFFECTIVE_END_DATE = '{D}' WHERE UNIQUE_ID = '{I}'".format(D = eff_end_date, I = i)

#   mycursor = mydb.cursor()
#   mycursor.execute(sql, multi=True)
#   mydb.commit()
#   mycursor.close()

In [ ]:
# Redeemed

In [ ]:
column.update({
    "PAYMENT": "RECEIVED PAYMENT TYPE",
    "DATE_REDEEMED": "RECEIVED PAYMENT EFFECTIVE DATE",	
    "CHECK_AMOUNT": "RECEIVED PAYMENT CHECK AMOUNT",
    "CHECK_NUMBER": "RECEIVED PAYMENT CHECK NUMBER",
    "CHECK_RECEIVED": "RECEIVED PAYMENT CHECK RECEIVED",
    "SOURCE": "RECEIVED PAYMENT SOURCE",
    "METHOD" : "RECEIVED PAYMENT METHOD",
    "DESCRIPTION": "RECEIVED PAYMENT DESCRIPTION"
})


column_dict = dict((v,k) for k,v in column.items())
req_cols = list(column_dict.values())

df3 = df.copy()
df3 = df3.rename(columns=column_dict)

# df3 = df3[df3['PARCEL_ID'] == '10 00263-0000-00029']
df3 = df3[df3['PAYMENT'] == 'Redemption Check']

df3 = df3[['UNIQUE_ID', 'DATE_REDEEMED', 'CHECK_AMOUNT', 'CHECK_NUMBER', 'CHECK_RECEIVED', 'SOURCE', 'METHOD', 'DESCRIPTION']]

df3['DATE_REDEEMED'] = pd.to_datetime(df3['DATE_REDEEMED']).dt.strftime("%Y-%m-%d").astype(str)
df3['CHECK_RECEIVED'] = pd.to_datetime(df3['CHECK_RECEIVED']).dt.strftime("%Y-%m-%d").astype(str)
df3['DESCRIPTION'] = df2['DESCRIPTION'].astype(str)
df3['DESCRIPTION'] = df2['DESCRIPTION'].replace('nan','')
df3['DESCRIPTION'] = df2['DESCRIPTION'].replace('NaN','')


df3.to_sql('REDEEM', engine, if_exists='append', index=False)




/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:1685: UserWarning: The provided table name 'REDEEM' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
